## Limpieza de Datos

#### Limpieza de la variable Teléfono

In [1]:
import pandas as pd
df = pd.read_csv("all_data.csv", dtype={"TELEFONO": str})

In [2]:
# Cuántos valores únicos y cuántos vacíos
print(df["TELEFONO"].nunique(), "valores únicos")
print(df["TELEFONO"].isna().sum(), "NaN / vacíos")
print("Total rows:", df.shape[0])

4215 valores únicos
46 NaN / vacíos
Total rows: 6599


In [3]:
# Valores con notación científica (“.0”)  
mask_sci = df["TELEFONO"].str.contains(r"\.0$", na=False)
print("Con .0 al final:", mask_sci.sum())

# Valores con caracteres no numéricos  
mask_nonnumeric = df["TELEFONO"].str.contains(r"\D", na=False)
print("Con otros símbolos:", mask_nonnumeric.sum())

# Longitudes  
longitudes = df["TELEFONO"].dropna().str.replace(r"\D", "", regex=True).str.len()
print(longitudes.describe())


Con .0 al final: 3522
Con otros símbolos: 3556
count    6553.000000
mean        8.577598
std         0.782789
min         2.000000
25%         8.000000
50%         9.000000
75%         9.000000
max        24.000000
Name: TELEFONO, dtype: float64


In [4]:
import re

def limpiar_telefono(x):
    if pd.isna(x):
        return "-1"  # o "Sin teléfono"

    s = str(x).strip()

    # Handle numbers in scientific notation
    if 'e' in s or 'E' in s:
        try:
            s = str(int(float(s)))
        except Exception:
            return "-1"

    # 1. Eliminar todo lo que no sea dígito
    s = re.sub(r"\D+", "", s)

    # 2. Si quedó vacío
    if not s:
        return "-1"

    # 3. Tomar solo el primer número si había varios concatenados
    if len(s) > 8:
        s = s[:8]

    # 4. Validar longitud mínima
    if len(s) < 7:
        return "-1"

    # 5. Para asegurar formato de 8 dígitos:  
    s = s.zfill(8)

    return s

In [5]:
df["TELEFONO_LIMPIO"] = df["TELEFONO"].apply(limpiar_telefono)

In [6]:
# ¿Cuántos inválidos quedan?
print((df["TELEFONO_LIMPIO"] == "-1").sum(), "invalidos")

# Histograma de longitudes (deberían ser todos 8 o el marcador -1)
long_post = df["TELEFONO_LIMPIO"].str.len().value_counts()
print(long_post)

51 invalidos
TELEFONO_LIMPIO
8    6548
2      51
Name: count, dtype: int64
